In [1]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments
from datasets import load_dataset

c:\Users\maria\miniconda3\envs\machine_learning_py38\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 1. Load and Filter the Dataset
Load the Gigaword dataset and filter for summaries with a maximum of 5 words

In [2]:
dataset = load_dataset("gigaword")

# Define a filter function to keep only summaries with 5 words or fewer
def filter_short_summaries(example):
    return len(example["summary"].split()) <= 5

# Apply the filter to the training and validation sets
train_data = dataset["train"].filter(filter_short_summaries)
validation_data = dataset["validation"].filter(filter_short_summaries)

# Check dataset sizes after filtering
print(f"Filtered train dataset size: {len(train_data)}")
print(f"Filtered validation dataset size: {len(validation_data)}")

c:\Users\maria\miniconda3\envs\machine_learning_py38\lib\site-packages\datasets\load.py:1486: FutureWarning: The repository for gigaword contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/gigaword
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Filtered train dataset size: 434484
Filtered validation dataset size: 19570


# 2. Preprocess the Data for T5
Define a preprocessing function to add the "summarize:" prefix and tokenize

In [3]:
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)

def preprocess_data(batch):
    # Add "summarize:" prefix to each input text in the batch
    input_texts = ["summarize: " + doc for doc in batch['document']]
    target_texts = batch['summary']
    
    # Tokenize inputs and labels
    inputs = tokenizer(input_texts, max_length=50, truncation=True, padding="max_length")
    targets = tokenizer(target_texts, max_length=5, truncation=True, padding="max_length")
    
    # Return tokenized input and labels
    inputs['labels'] = targets['input_ids']
    return inputs

# Apply the preprocessing to the train and validation sets
train_data = train_data.map(preprocess_data, batched=True, remove_columns=["document", "summary"])
validation_data = validation_data.map(preprocess_data, batched=True, remove_columns=["document", "summary"])

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


# 3. Initialize the Model

In [4]:
model = T5ForConditionalGeneration.from_pretrained(model_name)

# 4. Define Training Arguments

In [5]:
training_args = TrainingArguments(
    output_dir="./t5_summarization",
    per_device_train_batch_size=32,      # Adjust based on your GPU memory
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    learning_rate=1e-3,
    warmup_steps=500,
    weight_decay=0.01,
    logging_steps=100,
    save_total_limit=2,
    report_to='none',
    disable_tqdm=False,
    fp16=True  # Enable mixed precision for faster training if supported
)

# 5. Initialize Trainer

In [6]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=validation_data,
)

# 6. Train the Model from Scratch

In [7]:
trainer.train()

  0%|          | 101/135780 [00:22<6:56:58,  5.42it/s]

{'loss': 3.5472, 'grad_norm': 3.307152032852173, 'learning_rate': 0.000194, 'epoch': 0.01}


  0%|          | 201/135780 [00:40<6:59:25,  5.39it/s]

{'loss': 2.3227, 'grad_norm': 3.4373724460601807, 'learning_rate': 0.00039400000000000004, 'epoch': 0.01}


  0%|          | 301/135780 [00:59<7:00:51,  5.37it/s]

...

                                                         
100%|██████████| 135780/135780 [7:08:32<00:00,  5.19it/s]

{'eval_loss': 1.395562767982483, 'eval_runtime': 135.871, 'eval_samples_per_second': 144.034, 'eval_steps_per_second': 18.01, 'epoch': 10.0}


100%|██████████| 135780/135780 [7:08:41<00:00,  5.28it/s]

{'train_runtime': 25721.0414, 'train_samples_per_second': 168.922, 'train_steps_per_second': 5.279, 'train_loss': 1.0993432358660145, 'epoch': 10.0}


TrainOutput(global_step=135780, training_loss=1.0993432358660145, metrics={'train_runtime': 25721.0414, 'train_samples_per_second': 168.922, 'train_steps_per_second': 5.279, 'total_flos': 5.7425632100352e+16, 'train_loss': 1.0993432358660145, 'epoch': 10.0})

# 7. Evaluate the Model
Run predictions on a small batch to verify results

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

sample = validation_data.shuffle(seed=42).select(range(5))
for example in sample:
    # Move input IDs to the same device as the model
    input_ids = torch.tensor(example['input_ids']).unsqueeze(0).to(device)
    
    # Generate predictions using the model on the same device
    predicted_summary_ids = model.generate(input_ids, max_length=5)
    
    # Decode input and output for readability
    input_text = tokenizer.decode(example['input_ids'], skip_special_tokens=True)
    reference_summary = tokenizer.decode(example['labels'], skip_special_tokens=True)
    predicted_summary = tokenizer.decode(predicted_summary_ids[0], skip_special_tokens=True)

    # Print the results
    print(f"\nInput: {input_text}")
    print(f"Reference Summary: {reference_summary}")
    print(f"Predicted Summary: {predicted_summary}")


Input: summarize: coastal shandong province plans to readjust its traditional export structure this year in a bid to improve export quality.
Reference Summary: shandong
Predicted Summary: shandong

Input: summarize: still skeptical about the stock market's long-term potential, investors cashed in some gains from two weeks of rallies tuesday, sending prices lower.
Reference Summary: results from s
Predicted Summary: stocks fall in early

Input: summarize: hong kong stocks fell ###.## points, or #.## percent, to close at ##,###.## points wednesday.
Reference Summary: hong kong
Predicted Summary: hong kong

Input: summarize: two momentous civil war battles were fought on the rolling virginia countryside just north of a little village called manassas junction, waged ## months apart and each time changing the course of
Reference Summary: history traffic battle at
Predicted Summary: battles rage

Input: summarize: monaco's togolese international striker emmanuel adebayor has insisted he is 

# 8. Save the Fine-Tuned Model and Tokenizer

In [10]:
model.save_pretrained("./t5_summarization_model")
tokenizer.save_pretrained("./t5_summarization_model")

('./t5_summarization_model\\tokenizer_config.json',
 './t5_summarization_model\\special_tokens_map.json',
 './t5_summarization_model\\spiece.model',
 './t5_summarization_model\\added_tokens.json')